In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from tqdm import tqdm
import sys
import matplotlib as mpl

sys.modules.pop('generate_syn_data', None)
from generate_syn_data import *

sys.modules.pop('ARW', None)
from ARW import *

In [ ]:
trials_per_var = 20
num_periods = 50
alpha = 0.1; delta = 0.1; gamma=1
fixed_windows = [1, 64]

#stationary case
true_means = 10 * np.ones(num_periods)

#NOTE: uncomment to change cases

#non-stationary case
true_means = generate_true_means(num_periods-1, 2)
#true_means = np.sin(np.linspace(0, 8*np.pi, num_periods)) + 5

plt.plot(true_means)

B_arr = np.random.randint(low=5, high=10, size=num_periods)
B_arr_starts = np.cumsum(B_arr) - B_arr
B_arr_ends = np.cumsum(B_arr) - 1

#seeds for the trials
variances = [0.01, 0.1, 1]
seeds = np.arange(len(variances) * trials_per_var)

#initialize dictionary for storing the results
CDF_dict = {}
CDF_dict['ARW'] = {}
for k in fixed_windows:
    CDF_dict[f'Val_{k}'] = {}
    for variance in variances:
        CDF_dict[f'Val_{k}'][variance] = {}
        CDF_dict['ARW'][variance] = {}
        for trial in range(trials_per_var):
            CDF_dict[f'Val_{k}'][variance][trial] = []
            CDF_dict['ARW'][variance][trial] = []

for (trial, seed) in tqdm(enumerate(seeds)):

    np.random.seed(seed)
    variance = variances[trial // trials_per_var]

    U = generate_gaussian_sequence(B_arr = B_arr, true_means = true_means, variance=variance)

    #k_hat_all = []
    #true_cdfs = np.zeros(( num_periods, 1+len(fixed_windows)))
    
    for t in range(num_periods):
            
        mu_t = true_means[t]

        U_t = U[:B_arr_ends[t]+1]
        B_arr_t = B_arr[:t+1]

        khat, qt_khat, qtk_all = ARWQE(U_t, B_arr_t, alpha, delta, gamma)
        #k_hat_all.append(khat)

        CDF_dict['ARW'][variance][trial % trials_per_var].append(norm.cdf(qt_khat, mu_t, np.sqrt(variance)))

        #baseline: take quantile of fixed k
        for ik, k in enumerate(fixed_windows):
            CDF_dict[f'Val_{k}'][variance][trial % trials_per_var].append(norm.cdf(qtk_all[min(k,t)], mu_t, np.sqrt(variance)))
            
    #k_hat_all = np.array(k_hat_all)

In [246]:
#process results
methods = ['ARW'] + [f'Val_{k}' for k in fixed_windows]
num_methods = len(methods)

#save result dictionary to numpy array
CDF_array = np.zeros((len(methods), len(variances), trials_per_var, num_periods))
rel_error = np.zeros((len(methods), len(variances), trials_per_var, num_periods))
for (i, key) in enumerate(CDF_dict.keys()):
    for (j, variance) in enumerate(CDF_dict[key].keys()):
        for (trial, trial_CDF) in CDF_dict[key][variance].items():
            #record square errors 
            CDF_array[i, j, trial, :] = trial_CDF

#take average of cdf over time and trials for each variance (n_model * n_variance)
cdf_mean_over_variances = np.mean(np.mean(CDF_array, axis=3), axis=2)

#take average of cdf over for each trial (n_model * n_variance * n_periods)
cdf_mean_over_trials = np.mean(CDF_array, axis=2)


In [ ]:
#plot results
colors = ['r', '#FFA500', 'tab:purple', 'tab:brown', 'tab:green', '#0096FF']

for (idx, variance) in enumerate(variances):
    print(f'Variance: {variance}')
    #for each variance, barplot mean MSE of each method
    fig, ax = plt.subplots(1, 1, figsize = (3, 3))    
    ax.bar(range(num_methods), cdf_mean_over_variances[:, idx], color=colors, \
           width = 0.8)
    #show number on yaxis
    for i, v in enumerate(cdf_mean_over_variances[:, idx]):
        ax.text(i - 0.1, v + 0.01, f'{v:.2f}', color='black')
    ax.set_xlabel('Quantile Estimation Algorithms')
    ax.set_xticks(range(num_methods))
    ax.set_xticklabels(['ARW', 'V1','V256'])
    ax.set_ylabel(r'Mean CDF, $F_t(\widehat{q}_{t,\hat{k}})$')
    mpl.rcParams.update({'font.size': 10})
    plt.tight_layout()
    plt.show()
    
    #uncomment to save the figure
    #plt.savefig(f'{DATAPATH}/{figname}_var{variance}.png', dpi=300, bbox_inches='tight')

In [ ]:
#plot cdf over time periods for ARW, V1, V256

for v, variance in enumerate(variances):

    fig, ax = plt.subplots(1, 1, figsize = (6,3))

    #ARW
    plt.plot(cdf_mean_over_trials[0][v], color='r', label='ARW')

    #Val_1
    plt.plot(cdf_mean_over_trials[1][v], color='#FFA500', label='V1')

    #val_256
    plt.plot(cdf_mean_over_trials[-1][v], color='#0096FF', label='V256')

    #plot nominal level
    plt.plot([0, num_periods], [1-alpha, 1-alpha], 'k--', label='Nominal Level')
    #figure settings
    mpl.rcParams.update({'font.size': 12})
    plt.tight_layout()
    plt.ylabel(r'CDF, $F_t(\widehat{q}_{t,\hat{k}})$')
    plt.xlabel('Time')

    #uncomment to show legend and save the figure

    plt.legend()
    #plt.savefig(f'{DATAPATH}/{figname}_time_var{variance}.png', dpi=300, bbox_inches='tight')
